# Mnist classification demo

In [1]:
%%capture
import numpy as np
import keras
from keras.datasets import mnist
from keras import backend as K
import os

## 1. Read data sample

We use a standard data, supplied by the `keras.datasets` library.

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_rows, img_cols = 28, 28
num_classes = 10

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## 2. Perform inference

We assume that you have already deployed the model as an application.

In [3]:
from hydrosdk import Cluster, Application

In [13]:
cluster = Cluster(
    http_address="<hydrosphere-http-address>",
    grpc_address="<hydrosphere-grpc-address>", 
    ssl=True,                                          # turn off, if your Hydrosphere instance doesn't have
    grpc_credentials=grpc.ssl_channel_credentials()    # TLS certificates installed
)

In [15]:
application = Application.find(cluster, "<application-name>")
application.lock_while_starting()
predictor = application.predictor()

Take a sample of first 10 records from the testing set.

In [37]:
sample = x_test[:10,].astype("double")

Perform the actial inference.

In [38]:
result = predictor.predict({"images": sample})

Compare results with the ground truth data.

In [46]:
result["probabilities"].argmax(axis=1)

array([7, 2, 1, 0, 4, 1, 4, 9, 6, 9])

In [45]:
y_test[:10,].argmax(axis=1)

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9])